In [4]:
from math import *
import numpy as np
import cmath
import matplotlib.pyplot as plt


In [5]:
# This import registers the 3D projection, but is otherwise unused.
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

In [6]:
def sinc(x):
    '''
    Compute the sinc-function: sin(x) / x
    '''
    try:  
        return sin(x) / float(x)
    except ZeroDivisionError:
        return 1.

In [10]:
def upperRecurency(value, Nmax):
    
    # Nmax is the larger value of requirement function but quantity is (Nmax + 1) cause 0 is included
    sphj, sphy, new_sphj = np.zeros(Nmax + 1), np.zeros(Nmax + 1), np.zeros(Nmax + 101)
    
    if value:
        
        # Compute the spherical Bessels function of the first kind
        sphj[0] = sinc(value)                                                       # n = 0
        sphj[1] = (-1) * cos(value) / float(value) + sin(value) / float(value ** 2) # n = 1
        
        # Compute the spherical Bessels function of the second kind
        sphy[0] = (-1) * cos(value) / float(value)                                  # n = 0
        sphy[1] = (-1) * sin(value / float(value) - cos(value) / float(value ** 2)) # n = 1  
        
#     else:
#         sphj.append('ZeroDivisionError')
        
    for ind in range(2, Nmax + 1):
        sphy[ind] = (2 * ind - 1) / float(value) * sphy[ind - 1] - sphy[ind - 2]
        
        if (ind <= value):
            sphj[ind] = (2 * ind - 1) / float(value) * sphj[ind - 1] - sphj[ind - 2]
    
    
    new_sphj[Nmax + 100] = 0 # n = Nmax + 101
    new_sphj[Nmax + 99] = 1 # n = Nmax + 100
    for ind in range(98 + Nmax, -1, -1):
        new_sphj[ind] = (2 * ind + 3) / float(value) * new_sphj[ind + 1] - new_sphj[ind + 2]
        if new_sphj[ind] > 100:
            if ind <= Nmax:
                for index in range(ind, Nmax, 1):
                    new_sphj[index] /= 100
            else:
                new_sphj[ind + 1] /= 100
                new_sphj[ind] /= 100
                
    
    for ind in range(len(sphj)):
        if sphj[ind] != 0:
            k = new_sphj[ind] / float(sphj[ind])
            break
            
    bessels = []
    for elem in new_sphj:
        bessels.append(elem / float(k))
#         print(elem / float(k))
    
#     for ind in range(len(sphj)):
#         if sphj[ind]:
#             print(f'top_to_bottom: {bessels[ind]}\n', f'bottom_to_top: {sphj[ind]}\n')

    return bessels, sphy
    
# for elem in range(1,1700):
#     print(upperRecurency(elem, 10))

In [ ]:
def sqrthcos(r, z0, costheta):
    return (r ** 2 + z0 ** 2 + 2 * r * z0 * costheta) ** .5

Nmax = 10
z0 = 50
step = 1
p_ith = np.zeros((401, 401), dtype=complex)
p_spth = np.zeros((401, 401), dtype=complex)
P = np.zeros(Nmax)
p_i = np.zeros((401, 401), dtype=complex)
for x in range(-200, 201):
    for z in range(-200, 201):
        r = step * (x ** 2 + z ** 2) ** .5
        if r:
            costheta = step * z / r
            if sqrthcos(r, z0, costheta):
                sphj, sphy = upperRecurency(r, Nmax)
                p_ith[x + 200][z + 200] = cmath.exp(1j * r * costheta)
                p_spth[x + 200][z + 200] = cmath.exp(1j * sqrthcos(r, z0, costheta)) / sqrthcos(r, z0, costheta)
                P[0] = 1
                P[1] = costheta
                for n in range(0, Nmax - 2):
                    P[n + 2] = (2 * n + 1) / (n + 1) * costheta * P[n + 1] - n / (n + 1) * P[n];
                    p_i[x + 200, z + 200] += (1j ** n)  * (2 * n + 1) * P[n] * sphj[n];
                print(p_i[x + 200, z + 200])

In [ ]:
print(abs(p_i[x+200,z+200]-p_ith[x+200, z+200]))